In [22]:
%matplotlib inline

import configparser
import os

import requests
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse, stats, spatial
import scipy.sparse.linalg
from sklearn import preprocessing, decomposition
import librosa
import IPython.display as ipd
import json
import tmdbsimple as tmdb
from itertools import chain

In [23]:
tmdb.API_KEY = 'b198753b805906fe47aea13d301de22d'

In [24]:
dataset = pd.read_csv('Saved_Datasets/NewFeaturesDataset.csv')

In [25]:
dataset.head(5)

,id,budget,genres,imdb_id,overview,production_companies,release_date,revenue,title,director_name,actor_names,actors_ids,actors_tenures,total_tenure,average_tenure,Metacritic,ROI,success
0,12,94000000,Animation|Family,266543,"Nemo, an adventurous young clownfish, is unexp...",Pixar Animation Studios,2003-05-30,940335536,Finding Nemo,Andrew Stanton,"['Albert Brooks', 'Ellen DeGeneres', 'Alexande...","[14, 5293, 12, 13, 18]","[18, 24, 2, 28, 14]",86,17.20,90,2.639,4
1,16,12800000,Drama|Crime|Music,168629,"Selma, a Czech immigrant on the verge of blind...",Fine Line Features,2000-05-17,40031879,Dancer in the Dark,Lars von Trier,"['Björk', 'Catherine Deneuve', 'David Morse', ...","[6748, 47, 52, 50, 53]","[49, 19, 21, 44, 15]",148,29.60,61,2.127,3
2,22,140000000,Adventure|Fantasy|Action,325980,"Jack Sparrow, a freewheeling 17th-century pira...",Walt Disney Pictures,2003-09-07,655011224,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,"['Johnny Depp', 'Geoffrey Rush', 'Orlando Bloo...","[1709, 116, 114, 118, 85]","[7, 9, 7, 22, 20]",65,13.00,63,2.639,4
3,24,30000000,Action|Crime,266697,An assassin is shot at the altar by her ruthle...,Miramax Films,2003-10-10,180949000,Kill Bill: Vol. 1,Quentin Tarantino,"['Uma Thurman', 'Lucy Liu', 'Vivica A. Fox', '...","[140, 589, 2535, 139, 141]","[12, 26, 15, 17, 39]",109,21.80,69,2.639,4
4,25,72000000,Drama|War,418763,Jarhead is a film about a US Marine Anthony Sw...,Universal Pictures,2005-04-11,96889998,Jarhead,Sam Mendes,"['Jamie Foxx', 'Scott MacDonald', 'Lucas Black...","[133, 134, 1350483, 155]","[11, 14, 0, 12]",37,9.25,58,0.346,2


## 1.Forming data and actor list extraction ###

In [26]:
#Forming data in column actor_names
print(dataset['actor_names'][5])
dataset['actor_names']=dataset['actor_names'].str.replace('"','')
print(dataset['actor_names'][5])
dataset['actor_names'] = dataset['actor_names'].str.replace("'",'')
print(dataset['actor_names'][5])
dataset['actor_names'] = dataset['actor_names'].str.replace("[",'')
dataset['actor_names'] = dataset['actor_names'].str.replace("]",'')
print(dataset['actor_names'][5])
dataset['actor_names'] = dataset['actor_names'].str.replace(", ",',')
print(dataset['actor_names'][5])

['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey']
['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey']
[Kieran OBrien, Margo Stilley, Courtney Taylor-Taylor, Alex Kapranos, Guy Garvey]
Kieran OBrien, Margo Stilley, Courtney Taylor-Taylor, Alex Kapranos, Guy Garvey
Kieran OBrien,Margo Stilley,Courtney Taylor-Taylor,Alex Kapranos,Guy Garvey


In [27]:
#Example line
test = dataset['actor_names'][5].split(",")
print(test)


['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey']


In [28]:
#Compute movies-actor dictionnary and actor list
movies_actor = {}
actor_list = []
for i in range(0, len(dataset)):
    one_line = dataset['actor_names'][i].split(",")
    movies_actor[dataset['title'][i]] = one_line
    #movie_name = dataset['title']
    actor_list.extend(one_line)  

In [29]:
print(actor_list)

['Albert Brooks', 'Ellen DeGeneres', 'Alexander Gould', 'Willem Dafoe', 'Brad Garrett', 'Björk', 'Catherine Deneuve', 'David Morse', 'Peter Stormare', 'Joel Grey', 'Johnny Depp', 'Geoffrey Rush', 'Orlando Bloom', 'Keira Knightley', 'Jack Davenport', 'Uma Thurman', 'Lucy Liu', 'Vivica A. Fox', 'Daryl Hannah', 'David Carradine', 'Jamie Foxx', 'Scott MacDonald', 'Lucas Black', 'Peter Sarsgaard', 'Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey', 'Dan Castellaneta', 'Julie Kavner', 'Nancy Cartwright', 'Yeardley Smith', 'Hank Azaria', 'Jim Carrey', 'Kate Winslet', 'Kirsten Dunst', 'Mark Ruffalo', 'Elijah Wood', 'Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Bill Nighy', 'Viggo Mortensen', 'Maria Bello', 'Heidi Hayes', 'Ashton Holmes', 'William Hurt', 'Eminem', 'Mekhi Phifer', 'Brittany Murphy', 'Kim Basinger', 'Michael Shannon', 'Joaquin Phoenix', 'Reese Witherspoon', 'Ginnifer Goodwin', 'Robert Patrick', 'Tyler Hilton', 'Cli

In [30]:
print(movies_actor)

{'Finding Nemo': ['Albert Brooks', 'Ellen DeGeneres', 'Alexander Gould', 'Willem Dafoe', 'Brad Garrett'], 'Dancer in the Dark': ['Björk', 'Catherine Deneuve', 'David Morse', 'Peter Stormare', 'Joel Grey'], 'Pirates of the Caribbean: The Curse of the Black Pearl': ['Johnny Depp', 'Geoffrey Rush', 'Orlando Bloom', 'Keira Knightley', 'Jack Davenport'], 'Kill Bill: Vol. 1': ['Uma Thurman', 'Lucy Liu', 'Vivica A. Fox', 'Daryl Hannah', 'David Carradine'], 'Jarhead': ['Jamie Foxx', 'Scott MacDonald', 'Lucas Black', 'Peter Sarsgaard'], '9 Songs': ['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey'], 'The Simpsons Movie': ['Dan Castellaneta', 'Julie Kavner', 'Nancy Cartwright', 'Yeardley Smith', 'Hank Azaria'], 'Eternal Sunshine of the Spotless Mind': ['Jim Carrey', 'Kate Winslet', 'Kirsten Dunst', 'Mark Ruffalo', 'Elijah Wood'], "Pirates of the Caribbean: Dead Man's Chest": ['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Bill Nig

In [31]:
print(len(actor_list))
actor_list = list(set(actor_list))
print(len(actor_list))
#print(list(set(test)))

12925
4644


In [34]:
#for movie, actor in movies_actor.items():
actor_movies = {}
for actor in actor_list:
    movies_of_actor = []
    for i in range(0,len(movies_actor.keys())):
        if actor in list(movies_actor.values())[i]:
            movies_of_actor.append(list(movies_actor.keys())[i])
    actor_movies[actor] = movies_of_actor        

In [280]:
print(actor_movies)

{'Vicky Haughton': ['Whale Rider'], 'Jeremy Irons': ['Kingdom of Heaven', 'Eragon', 'Dungeons & Dragons', 'Appaloosa', 'Margin Call', 'The Words', 'Beautiful Creatures', "Assassin's Creed", 'Race'], 'Leslie Mann': ['Knocked Up', 'I Love You Phillip Morris', '17 Again', 'Funny People', 'Rio', 'The Change-Up', 'The Other Woman', 'This Is 40', 'Rio 2'], 'Sean Maguire': ['Meet the Spartans'], 'David Mann': ['Meet The Browns'], 'Paddy Considine': ['Cinderella Man', 'The Bourne Ultimatum', 'Hot Fuzz', "The World's End", 'Child 44', 'Macbeth'], 'Tang Wei': ['Blackhat'], 'Elyes Gabel': ['A Most Violent Year'], 'Ralph Garman': ['Tusk'], 'Matthew Gray Gubler': ['(500) Days of Summer', 'Alvin and the Chipmunks: The Squeakquel'], 'Ben Winchell': ['Max Steel'], 'Bruno Bichir': ['Julia'], 'Maura Tierney': ['Insomnia', 'Welcome to Mooseport'], 'Jemima West': ['The Mortal Instruments: City of Bones'], 'Ian Gomez': ['My Big Fat Greek Wedding 2'], 'Danny Glover': ['Saw', 'Dreamgirls', 'Death at a Funera

In [281]:
Actors_mov = pd.DataFrame()
Actors_mov['movies in dataset'] = actor_movies

#Actors_mov.to_csv('Saved_Datasets/Actors_mov.csv', index=False)

### 2. Actors data collection with TMDb API ###

In [19]:
#Get tmdb ids of actors
def get_person_id(actors_list):
    name_error = []
    search = tmdb.Search()
    #person_id = 0
    try:
        person_query = search.person(query=actors_list)
        person_id = person_query['results'][0]['id']
    except:
        person_id = -1
        name_error.append(actors_list)
    return person_id, name_error

   
# Search for people credits
def get_people_credit (person_id):
    #Get people credits
    people = tmdb.People(person_id)
    credits = people.movie_credits()
    credits_cast = credits['cast']
    return credits['cast']    

#Compute people tenure
def get_people_tenure (person_id):
    #print(person_id)
    credits = get_people_credit (person_id)
    nb_movies = len(credits)
    #movie_range = np.arange(nb_movies)
    if credits != []:
        years=[]
        for i in range(0,nb_movies):
            try: #to catch some unexplicable errors
                year_movie = credits[i]['release_date'][0:4] 
            except:
                continue
            if (year_movie < '2017') & (year_movie != ''):
                years.append(year_movie)
        years = sorted(years) 
   
        if years != []:
            date = [years[0],years[len(years)-1]]
            tenure = (int(years[int(len(years))-1])) - int(years[0])+1
        else:
            date = []
            tenure = 0
    else:
        date = []
        tenure = 0 
    return date, tenure, nb_movies

In [14]:
i = actor_list

# Compute person ids
err_idd = []
for x in map(get_person_id,i):
    #print(x)
    err_idd.append(x)

#Extract good ids and bad ids in separated list    
ids = []
err_name = []
for j in range (0,len(err_idd)):
    nb, err = err_idd[j]
    ids.append(nb)
    err_name.append(err)

In [15]:
print(ids)
print(err_name) 

[15295, 16940, 41087, 52262, 9445, 14887, 38280, 78050, 60286, 5661, 997369, 52745, 16307, 1075355, 34457, 2047, 199, 30326, 77278, 9443, 87822, 230818, 59251, 116278, 33192, 71535, 58370, 55086, 84497, 1166, 972306, 54711, 1304662, 63864, 16273, 16501, 9594, 6217, 1234, 33684, 25837, 2165, 1256170, 18746, 1339, 11805, 5823, 74036, 2222, 17198, 44301, 60072, 34202, 17287, 55900, 17288, 53493, 16867, 5695, 28638, 81260, 81682, 37048, 90596, 55536, 934219, 32683, 46919, 4173, 205300, 58224, 1345230, 20176, 6161, 167132, 40275, 9560, 75036, 66431, 12536, 1047642, 64066, 32799, 76789, 35705, 140, 9785, 1278721, 1695766, 1111713, 19152, 66257, 27855, 37158, 19754, 1079805, 20879, 19974, 73889, 7470, 1056523, 1719927, 2390, 2598, 42363, 41819, 111946, 120248, 553504, 822, 78962, 16971, 31162, 168974, 966306, 23658, 1005852, 98394, 90325, 2839, 111922, 29221, 130640, 2203, 23628, 58431, 80378, 1711, 97303, 17142, 27858, 142965, 3977, 119589, 205474, 18271, 55466, 580648, 52891, 5216, 43858, 1

In [17]:
Actors_ids = pd.DataFrame()
Actors_ids['id'] = ids

#Actors_ids.to_csv('Saved_Datasets/Actors_ids_new.csv', index=False)

In [18]:
#Filter bad ids 
bad_ids_index = [k for k,y in enumerate(ids) if y == -1]
print(bad_ids_index)

good_ids = list(filter(lambda a: a != -1, ids))

print(good_ids)
print(len(good_ids))
print(len(ids))

[1003, 1026, 1372, 1820, 2001, 2647, 3863]
[15295, 16940, 41087, 52262, 9445, 14887, 38280, 78050, 60286, 5661, 997369, 52745, 16307, 1075355, 34457, 2047, 199, 30326, 77278, 9443, 87822, 230818, 59251, 116278, 33192, 71535, 58370, 55086, 84497, 1166, 972306, 54711, 1304662, 63864, 16273, 16501, 9594, 6217, 1234, 33684, 25837, 2165, 1256170, 18746, 1339, 11805, 5823, 74036, 2222, 17198, 44301, 60072, 34202, 17287, 55900, 17288, 53493, 16867, 5695, 28638, 81260, 81682, 37048, 90596, 55536, 934219, 32683, 46919, 4173, 205300, 58224, 1345230, 20176, 6161, 167132, 40275, 9560, 75036, 66431, 12536, 1047642, 64066, 32799, 76789, 35705, 140, 9785, 1278721, 1695766, 1111713, 19152, 66257, 27855, 37158, 19754, 1079805, 20879, 19974, 73889, 7470, 1056523, 1719927, 2390, 2598, 42363, 41819, 111946, 120248, 553504, 822, 78962, 16971, 31162, 168974, 966306, 23658, 1005852, 98394, 90325, 2839, 111922, 29221, 130640, 2203, 23628, 58431, 80378, 1711, 97303, 17142, 27858, 142965, 3977, 119589, 205474, 

In [32]:
#Compute dates and tenures of people
ten_date_list = []

for x in map(get_people_tenure,good_ids):
    ten_date_list.append(x)
  

In [54]:
print(len(ten_date_list))

4637


In [55]:
#Extract dates and tenures in separated list
dates = []
tenures = []
nb_movies = []
for i in range (0,len(ten_date_list)):
    dat, ten, nb = ten_date_list[i]
    dates.append(dat)
    tenures.append(ten)
    nb_movies.append(nb)

In [56]:
print(dates)
print(tenures)
print(nb_movies)

[['2000', '2010'], ['1974', '2016'], ['1996', '2016'], ['2000', '2015'], [], ['2000', '2015'], ['2006', '2016'], ['2008', '2015'], ['2005', '2016'], ['2004', '2016'], ['2010', '2016'], ['1989', '2015'], ['1987', '2012'], ['1999', '2015'], ['1997', '2016'], ['1979', '2016'], ['1975', '2011'], ['1993', '2007'], ['2007', '2015'], ['1971', '2016'], ['1994', '2016'], ['2002', '2010'], ['1999', '2015'], ['2002', '2016'], ['1996', '2016'], ['1995', '2016'], ['2006', '2016'], ['2008', '2016'], ['2000', '2016'], ['1970', '2016'], ['2009', '2009'], ['1993', '2016'], ['2012', '2015'], ['2006', '2007'], ['1977', '2016'], ['1997', '2016'], ['1965', '2012'], ['1984', '2016'], ['1987', '2016'], ['1983', '2015'], ['1997', '2014'], ['1978', '2016'], ['2015', '2015'], ['1964', '2016'], ['1996', '2016'], ['1987', '2016'], ['1956', '2015'], ['1991', '2016'], ['1948', '2016'], ['1998', '2015'], ['1974', '2011'], ['1997', '2016'], ['2008', '2015'], ['1995', '2016'], ['2006', '2016'], ['2001', '2016'], ['200

In [57]:
#Insert bad ids index into dates and tenures lists
for i in range(0,len(bad_ids_index)):
    dates.insert(bad_ids_index[i],0)
    tenures.insert(bad_ids_index[i],0)
    nb_movies.insert(bad_ids_index[i],0)

#Replacing some elements
tenures = [0 if x==[] else x for x in tenures]
print(tenures)

[11, 43, 21, 16, 0, 16, 11, 8, 12, 13, 7, 27, 26, 17, 20, 38, 37, 15, 9, 46, 23, 9, 17, 15, 21, 22, 11, 9, 17, 47, 1, 24, 4, 2, 40, 20, 48, 33, 30, 33, 18, 39, 1, 53, 21, 30, 60, 26, 69, 18, 38, 20, 8, 22, 11, 16, 16, 19, 57, 19, 10, 17, 1, 9, 19, 6, 21, 35, 50, 3, 19, 1, 18, 33, 6, 30, 53, 4, 11, 43, 79, 10, 23, 19, 18, 25, 36, 1, 1, 1, 31, 1, 15, 31, 37, 6, 34, 27, 6, 42, 9, 1, 38, 26, 35, 17, 18, 7, 6, 28, 14, 32, 27, 1, 6, 16, 7, 11, 15, 21, 8, 12, 8, 26, 28, 5, 7, 17, 5, 17, 23, 34, 40, 7, 7, 21, 19, 7, 17, 18, 17, 43, 10, 22, 38, 8, 11, 17, 12, 17, 26, 28, 39, 22, 10, 18, 19, 19, 5, 23, 21, 1, 39, 9, 63, 1, 20, 44, 27, 17, 16, 12, 1, 45, 40, 31, 8, 36, 21, 4, 33, 4, 19, 10, 19, 15, 30, 27, 1, 52, 7, 38, 13, 41, 11, 12, 43, 9, 49, 13, 10, 14, 10, 23, 46, 1, 1, 15, 6, 26, 22, 9, 19, 16, 37, 18, 28, 23, 1, 43, 18, 26, 22, 9, 46, 50, 21, 21, 1, 29, 4, 36, 1, 10, 18, 1, 27, 9, 25, 1, 12, 17, 17, 26, 22, 0, 33, 13, 18, 22, 16, 32, 1, 34, 19, 21, 25, 26, 1, 7, 62, 28, 1, 30, 25, 22, 18,

In [58]:
#Check to verify if bad ids index are well inserted
print(bad_ids_index)
for i in bad_ids_index:
    print(dates[i])

[1003, 1026, 1372, 1820, 2001, 2647, 3863]
0
0
0
0
0
0
0


## 3. Create and save Actors dataframe ###

In [60]:
Actors_ten_date_mov = pd.DataFrame()
Actors_ten_date_mov['dates'] = dates
Actors_ten_date_mov['tenures'] = tenures
Actors_ten_date_mov['nb movies'] = nb_movies

#Actors_ten_date_mov.to_csv('Saved_Datasets/Actors_ten_date_mov.csv', index=False)

In [341]:
print(len(actor_list))

#Creation of actors dataframe with general informations
Actors = pd.DataFrame()
Actors['tmdb_id'] = ids
Actors['Name'] = actor_list
Actors['date'] = dates
Actors['total_tenure']= tenures
Actors['nb_total_movies'] = nb_movies
Actors['movies_in_dataset'] = actor_movies.values()
Actors.head(10)

4644


,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset
0,15295,Vicky Haughton,"[2000, 2010]",11,5,[Whale Rider]
1,16940,Jeremy Irons,"[1974, 2016]",43,90,"[Kingdom of Heaven, Eragon, Dungeons & Dragons..."
2,41087,Leslie Mann,"[1996, 2016]",21,31,"[Knocked Up, I Love You Phillip Morris, 17 Aga..."
3,52262,Sean Maguire,"[2000, 2015]",16,7,[Meet the Spartans]
4,9445,David Mann,[],0,0,[Meet The Browns]
5,14887,Paddy Considine,"[2000, 2015]",16,44,"[Cinderella Man, The Bourne Ultimatum, Hot Fuz..."
6,38280,Tang Wei,"[2006, 2016]",11,15,[Blackhat]
7,78050,Elyes Gabel,"[2008, 2015]",8,7,[A Most Violent Year]
8,60286,Ralph Garman,"[2005, 2016]",12,16,[Tusk]
9,5661,Matthew Gray Gubler,"[2004, 2016]",13,27,"[(500) Days of Summer, Alvin and the Chipmunks..."


In [129]:
actor_range = np.arange(len(actor_list))


[0, 1813, 2080, 2315, 2538]
['2003', '2011', '2016', '2014', '2015']


### 3.1 Associate to actors the release dates of movies in which they have played

In [248]:
#Compute release dates of movies in which one actor has played.
def one_actor_movies_release_dates(actor):
    index_movie = [i for i, item in enumerate(dataset['title']) if item in Act_df['movies_in_dataset'][actor]]
    #print(index_movie)
    one_actor_movie_date = []
    for i in index_movie:
        one_actor_movie_date.append(dataset['release_date'][i][0:4])
    #print(one_actor_movie_date)    
    return one_actor_movie_date

In [287]:
test = one_actor_movies_release_dates(100)
print(test)

['2013']


In [288]:
actor_movies_realease_dat = list(map(one_actor_movies_release_dates,np.arange(len(actor_list))))

In [289]:
print(actor_movies_realease_dat)

[['2003'], ['2005', '2006', '2000', '2008', '2011', '2012', '2013', '2016', '2016'], ['2007', '2009', '2009', '2009', '2011', '2011', '2010', '2012', '2014', '2014'], ['2008'], ['2005'], ['2005', '2007', '2007', '2013', '2015', '2015'], ['2015'], ['2014'], ['2014'], ['2009', '2009'], ['2016'], ['2008'], ['2002', '2004'], ['2013'], ['2016'], ['2004', '2006', '2007', '2007', '2008', '2006', '2006', '2010', '2007', '2010', '2014'], ['2007'], ['2007'], ['2007', '2010'], ['2003'], ['2014', '2016'], ['2009'], ['2005', '2015'], ['2006'], ['2000', '2004', '2010', '2010', '2011', '2011', '2013', '2012', '2014', '2016', '2016', '2015', '2015', '2016'], ['2008'], ['2008', '2010', '2013', '2016'], ['2008', '2009', '2009', '2010', '2011', '2010', '2011', '2011'], ['2009', '2012', '2014', '2014', '2014', '2016', '2015'], ['2001', '2006'], ['2009'], ['2010'], ['2012'], ['2007'], ['2006'], ['2006', '2005', '2011'], ['2007'], ['2002'], ['2005'], ['2007'], ['2005', '2005', '2004', '2007'], ['2000', '201

In [342]:
Actors['Realease_date_of_movies_in_dataset'] = actor_movies_realease_dat

### 3.2 Compute for each movies, actors tenures until the release date of the movie

In [ ]:
#Compute for on movie actors tenures until the realease date of the movie
def get_tenure_actor_movie(actor):
    movies_tenure = []
    for i in range(0,len(Actors['Realease_date_of_movies_in_dataset'][actor])):
        if Actors['date'][actor] != []:
            if Actors['date'][actor]!= 0:
                movies_tenure.append((int(Actors['Realease_date_of_movies_in_dataset'][actor][i])-int(Actors['date'][actor][0]))+1)  
            else:
                movies_tenure.append(0)
        else:
            movies_tenure.append(0)
    return movies_tenure            

In [336]:
all_movies_tenure = list(map(get_tenure_actor_movie,np.arange(len(actor_list))))

In [343]:
Actors['Actors_tenure_in_movies'] = all_movies_tenure

In [344]:
Actors.head(10)

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies
0,15295,Vicky Haughton,"[2000, 2010]",11,5,[Whale Rider],[2003],[4]
1,16940,Jeremy Irons,"[1974, 2016]",43,90,"[Kingdom of Heaven, Eragon, Dungeons & Dragons...","[2005, 2006, 2000, 2008, 2011, 2012, 2013, 201...","[32, 33, 27, 35, 38, 39, 40, 43, 43]"
2,41087,Leslie Mann,"[1996, 2016]",21,31,"[Knocked Up, I Love You Phillip Morris, 17 Aga...","[2007, 2009, 2009, 2009, 2011, 2011, 2010, 201...","[12, 14, 14, 14, 16, 16, 15, 17, 19, 19]"
3,52262,Sean Maguire,"[2000, 2015]",16,7,[Meet the Spartans],[2008],[9]
4,9445,David Mann,[],0,0,[Meet The Browns],[2005],[0]
5,14887,Paddy Considine,"[2000, 2015]",16,44,"[Cinderella Man, The Bourne Ultimatum, Hot Fuz...","[2005, 2007, 2007, 2013, 2015, 2015]","[6, 8, 8, 14, 16, 16]"
6,38280,Tang Wei,"[2006, 2016]",11,15,[Blackhat],[2015],[10]
7,78050,Elyes Gabel,"[2008, 2015]",8,7,[A Most Violent Year],[2014],[7]
8,60286,Ralph Garman,"[2005, 2016]",12,16,[Tusk],[2014],[10]
9,5661,Matthew Gray Gubler,"[2004, 2016]",13,27,"[(500) Days of Summer, Alvin and the Chipmunks...","[2009, 2009]","[6, 6]"


In [345]:
#Actors.to_csv('Saved_Datasets/Actorsv3Dataset.csv', index=False)

### 3.3 Compute actors profitability###

In [43]:
Actors = pd.read_csv('Saved_Datasets/Actorsv3Dataset.csv')

In [22]:
Actors.head(1)

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies
0,15295,Vicky Haughton,"['2000', '2010']",11,5,['Whale Rider'],['2003'],[4]


In [108]:
dataset['actors_ids'] = dataset.actors_ids.apply(lambda x: x[1:-1].split(','))

In [110]:
act_ids = []
for i in range(0,len(dataset)):
    act = list(map(int,dataset['actors_ids'][i]))
    act_ids.append(act)

In [117]:
Act_ids =pd.DataFrame()
Act_ids ['ids'] = act_ids
Act_ids ['Profitability'] = dataset['Profitability']


In [119]:
Act_ids.head(2)

,ids,Profitability
0,"[14, 5293, 12, 13, 18]",846335536
1,"[6748, 47, 52, 50, 53]",27231879


In [91]:
dataset['actors_ids'] = dataset['actors_ids'].str.replace("'",'')

In [120]:
#Compute profitabtiliy of Actors
prof_actor = []
for i in Actors['tmdb_id']:
    prof_actor.append(Act_ids[Act_ids['ids'].apply({i}.issubset)]['Profitability'].sum())

In [123]:
Actors['Profitability'] = prof_actor

In [124]:
Actors.head(1)

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies,Profitability
0,15295,Vicky Haughton,"['2000', '2010']",11,5,['Whale Rider'],['2003'],[4],33400000


In [125]:
Actors.to_csv('Saved_Datasets/Actorsv4Dataset.csv', index=False)

## 4. Collect number of movie per actors until the release date of  movies in which he played ###

This section is not working, the function is almost functionnal but some errors are still not handled, for exemple the case for which two actors have the same name and the api of tmdb return the same id for the two actors, which is a problem.

In [144]:
Act_df = pd.read_csv('Saved_Datasets/Actorsv3Dataset.csv')

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies
128,97303,Pakak Innuksuk,"['2002', '2006']",5,2,['Atanarjuat: The Fast Runner'],['2002'],[1]
129,17142,Paul Dano,"['2000', '2016']",17,32,"['Little Miss Sunshine', 'There Will Be Blood'...","['2006', '2007', '2005', '2010', '2012', '2013...","[7, 8, 6, 11, 13, 14, 14, 17]"
130,27858,Adam Trese,"['1992', '2014']",23,12,['Silent House'],['2011'],[20]


In [300]:
Act_df = Act_df.drop(Act_df[Act_df['Name']=='Jr.'].index)

In [205]:
Act_df.iloc[0:4]

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies
0,15295,Vicky Haughton,"['2000', '2010']",11,5,['Whale Rider'],[2003],[4]
1,16940,Jeremy Irons,"['1974', '2016']",43,90,"['Kingdom of Heaven', 'Eragon', 'Dungeons & Dr...","[2005, 2006, 2000, 2008, 2011, 2012, 201...","[32, 33, 27, 35, 38, 39, 40, 43, 43]"
2,41087,Leslie Mann,"['1996', '2016']",21,31,"['Knocked Up', 'I Love You Phillip Morris', '1...","[2007, 2009, 2009, 2009, 2011, 2011, 201...","[12, 14, 14, 14, 16, 16, 15, 17, 19, 19]"
3,52262,Sean Maguire,"['2000', '2015']",16,7,['Meet the Spartans'],[2008],[9]


In [150]:
Act_df['Realease_date_of_movies_in_dataset'] = Act_df['Realease_date_of_movies_in_dataset'].str.replace("'",'')

In [151]:
Act_df['Realease_date_of_movies_in_dataset'] = Act_df.Realease_date_of_movies_in_dataset.apply(lambda x: x[1:-1].split(","))

In [114]:
print(Act_df['Realease_date_of_movies_in_dataset'][131])
print(Act_df['Realease_date_of_movies_in_dataset'][1])
mov_rel_date = []
for i in range(0,len(Act_df)):
    if Act_df['Realease_date_of_movies_in_dataset'][i] != ['']:
        mov_rel_date.append(list(map(int,Act_df['Realease_date_of_movies_in_dataset'][i])))
    else:
        mov_rel_date.append(-1)

['']
['2005', ' 2006', ' 2000', ' 2008', ' 2011', ' 2012', ' 2013', ' 2016', ' 2016']


In [117]:
print(mov_rel_date)

[[2003], [2005, 2006, 2000, 2008, 2011, 2012, 2013, 2016, 2016], [2007, 2009, 2009, 2009, 2011, 2011, 2010, 2012, 2014, 2014], [2008], [2005], [2005, 2007, 2007, 2013, 2015, 2015], [2015], [2014], [2014], [2009, 2009], [2016], [2008], [2002, 2004], [2013], [2016], [2004, 2006, 2007, 2007, 2008, 2006, 2006, 2010, 2007, 2010, 2014], [2007], [2007], [2007, 2010], [2003], [2014, 2016], [2009], [2005, 2015], [2006], [2000, 2004, 2010, 2010, 2011, 2011, 2013, 2012, 2014, 2016, 2016, 2015, 2015, 2016], [2008], [2008, 2010, 2013, 2016], [2008, 2009, 2009, 2010, 2011, 2010, 2011, 2011], [2009, 2012, 2014, 2014, 2014, 2016, 2015], [2001, 2006], [2009], [2010], [2012], [2007], [2006], [2006, 2005, 2011], [2007], [2002], [2005], [2007], [2005, 2005, 2004, 2007], [2000, 2010], [2015], [2004], [2005, 2001, 2009], [2007], [2004, 2007, 2001, 2010, 2004, 2010, 2010], [2003, 2004, 2012], [2008], [2006], [2011], [2007, 2008, 2006, 2011, 2011], [2008, 2008, 2010, 2011], [2007, 2006, 2004, 2001, 2008, 2010

In [154]:
print(Act_df['Realease_date_of_movies_in_dataset'][129])

['2006', ' 2007', ' 2005', ' 2010', ' 2012', ' 2013', ' 2013', ' 2016']


In [347]:
Act_df[Act_df['tmdb_id']== 65827]

,tmdb_id,Name,date,total_tenure,nb_total_movies,movies_in_dataset,Realease_date_of_movies_in_dataset,Actors_tenure_in_movies
488,65827,Keith David,"['1982', '2016']",35,136,"['The Princess and the Frog', 'Agent Cody Bank...","[2009, 2003, 2009]","[28, 22, 28]"
4040,65827,David Keith,"['1982', '2016']",35,136,"['U-571', 'Behind Enemy Lines', 'Raise Your Vo...","[2000, 2001, 2004]","[19, 20, 23]"


In [316]:
Act_df['tmdb_id'][488]

65827

In [416]:
dbl_id = []
for i in Act_df['tmdb_id']:
    if len(Act_df[Act_df['tmdb_id']== i]) > 1:
        dbl_id.append(list(Act_df[Act_df['tmdb_id']==i].index.values))       

In [417]:
print(dbl_id)

[[488, 4040], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1166, 1845], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1464, 3931], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1166, 1845], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1003, 1026, 1372, 1820, 2001, 2647, 3863], [1464, 3931], [488, 4040]]


In [472]:
diff_dbl_index = list(set(map(tuple,dbl_id)))
print(diff_dbl_index)

[(1003, 1026, 1372, 1820, 2001, 2647, 3863), (1464, 3931), (488, 4040), (1166, 1845)]


In [473]:
ind_keep = [x[0] for x in diff_dbl_index[1:]]
print(ind_keep)
ind_remove = list(set(ind_keep)-(set(diff_dbl_index[1:])))
print(ind_remove)

[1464, 488, 1166]
[1464, 488, 1166]


In [463]:
new_tmdb_id = Act_df['tmdb_id'].tolist()
new_tmdb_id=Act_df.drop(Act_df.index[ind_remove])
#new_tmdb_id = list(map(tmdb_id.remove,Act_df[tmdb_id[ind_remove]]))

In [468]:
print(new_tmdb_id['tmdb_id'][3931])
print(len(Act_df['tmdb_id']))

62
4643


In [433]:
#diff_dbl_index_l = []
#for x in diff_dbl_index[1:]:
#    for j in range(0,len(x)):
#        diff_dbl_index_l.append(x[j])
        
#print(diff_dbl_index_l)        

[1464, 3931, 488, 4040, 1166, 1845]


In [375]:
def get_one_nb_mov_for_mov(person_ids):
    ind = False
    nb_mov_dat = []
    cred_rel_date = []
    index_act = Act_df[Act_df.tmdb_id == person_ids].index.values
    dbl_id = []
    if len(index_act) > 1:
        index_act = [index_act][0]
   # print(person_ids)
   # print(index_act)
   # print(mov_rel_date[int(index_act)])
    if person_ids != -1 :
            if len(index_act) > 1:
                ind = True
                old_ind = index_act
                index_act = index_act[0]
            
            if mov_rel_date[int(index_act)] != -1:
                cred = get_people_credit (person_ids)
                print(person_ids)
                for j in range(0,len(cred)):
                    #print(cred[j])
                    try:
                        cred_rel_date.append(int(cred[j]['release_date'][0:4]))
                    except:
                        continue
                #print(cred_rel_date)    

                #print(mov_rel_date[int(index_act)])
                for i in mov_rel_date[int(index_act)]:
                    #print(i)
                    nb_mov = 0
                    c = len([k for k in cred_rel_date if k <= i])
                    #print(c)
                    nb_mov_dat.append(c)
                if ind == True: #Register double id to inset later 
                    for l in range(1,len(old_ind)):
                        print(l)
                        dbl_id.append(old_ind[l])
            else:
                nb_mov_dat.append(0)
    else:
            nb_mov_dat.append(0)
    return nb_mov_dat,dbl_id
        
    #c = len([i for i in act_df['release_date'][i] if i < t])

In [376]:
test,dlb = get_one_nb_mov_for_mov(65827)
print(test)
print(dlb)

65827
1
[96, 58, 96]
[4040]


In [307]:
nb_mov_dat_mov = list(map(get_one_nb_mov_for_mov,Act_df['tmdb_id']))

15295
16940
41087
52262
9445
14887
38280
78050
60286
5661
997369
52745
16307
1075355
34457
2047
199
30326
77278
9443
87822
230818
59251
116278
33192
71535
58370
55086
84497
1166
972306
54711
1304662
63864
16273
16501
9594
6217
1234
33684
25837
2165
1256170
18746
1339
11805
5823
74036
2222
17198
44301
60072
34202
17287
55900
17288
53493
16867
5695
28638
81260
81682
37048
90596
55536
934219
32683
46919
4173
205300
58224
1345230
20176
6161
167132
40275
9560
75036
66431
12536
1047642
64066
32799
76789
35705
140
9785
1278721
1695766
1111713
19152
66257
27855
37158
19754
1079805
20879
19974
73889
7470
1056523
1719927
2390
2598
42363
41819
111946
120248
553504
822
78962
16971
31162
168974
966306
23658
1005852
98394
90325
2839
111922
29221
130640
2203
23628
58431
80378
1711
97303
17142
27858
3977
119589
205474
18271
55466
580648
52891
5216
43858
13726
41309
25879
78018
22121
148096
31712
57192
8338
47296
18792
3266
486
87440
30315
17487
9462
590919
51992
1244
92019
589
83356
10254
146903
22044

TypeError: only length-1 arrays can be converted to Python scalars

In [232]:
print(nb_mov_dat_mov)

[[0], [7], [5], [6], [1], [11], [2, 3]]


In [ ]:
sum(i < 5 for i in j)
numpy.count_nonzero(lst < t)
c = len([i for i in lst if i < t])

In [50]:
print(Act_df[Act_df.tmdb_id == 15295 ].index.values)

<class 'pandas.core.indexes.numeric.Int64Index'>
[0]


In [258]:
#Compute movies-actor dictionnary and actor list
rr = {}
test = []
for i in range(0, len(dataset)):
    one_line = dataset['actor_names'][i].split(",")
    print(one_line)
    rr[dataset['title'][i]] = one_line
    #movie_name = dataset['title']
    test.extend(one_line) 

['Albert Brooks', 'Ellen DeGeneres', 'Alexander Gould', 'Willem Dafoe', 'Brad Garrett']
['Björk', 'Catherine Deneuve', 'David Morse', 'Peter Stormare', 'Joel Grey']
['Johnny Depp', 'Geoffrey Rush', 'Orlando Bloom', 'Keira Knightley', 'Jack Davenport']
['Uma Thurman', 'Lucy Liu', 'Vivica A. Fox', 'Daryl Hannah', 'David Carradine']
['Jamie Foxx', 'Scott MacDonald', 'Lucas Black', 'Peter Sarsgaard']
['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey']
['Dan Castellaneta', 'Julie Kavner', 'Nancy Cartwright', 'Yeardley Smith', 'Hank Azaria']
['Jim Carrey', 'Kate Winslet', 'Kirsten Dunst', 'Mark Ruffalo', 'Elijah Wood']
['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Bill Nighy']
['Viggo Mortensen', 'Maria Bello', 'Heidi Hayes', 'Ashton Holmes', 'William Hurt']
['Eminem', 'Mekhi Phifer', 'Brittany Murphy', 'Kim Basinger', 'Michael Shannon']
['Joaquin Phoenix', 'Reese Witherspoon', 'Ginnifer Goodwin', 'Robert Patrick', 'Tyler Hi

['Ashton Kutcher', 'Brittany Murphy', 'Christian Kane', 'David Moscow', 'Alex Thomas']
['Johnny Knoxville', 'Bam Margera', 'Steve-O', 'Chris Pontius', 'Ryan Dunn']
['Nicolas Cage', 'Adam Beach', 'Peter Stormare', 'Noah Emmerich', 'Mark Ruffalo']
['Michael Douglas', 'Sean Bean', 'Brittany Murphy', 'Skye McCole Bartusiak', 'Guy Torry']
['Eddie Murphy', 'Janet Jackson', 'Larry Miller', 'John Ales', 'Richard Gant']
['Leonardo DiCaprio', 'Russell Crowe', 'Mark Strong', 'Ali Suliman', 'Simon McBurney']
['Will Ferrell', 'John C. Reilly', 'Mary Steenburgen', 'Richard Jenkins', 'Adam Scott']
['Kurt Russell', 'Kevin Costner', 'Courteney Cox', 'Christian Slater', 'Kevin Pollak']
['Christian Slater', 'Tara Reid', 'Stephen Dorff', 'Frank C. Turner', 'Matthew Walker']
['Bill Paxton', 'Matthew McConaughey', 'Powers Boothe', 'Matt OLeary', 'Jeremy Sumpter']
['Shawn Wayans', 'Marlon Wayans', 'Jaime King', 'Frankie Faison', 'Maitland Ward']
['Mia Wasikowska', 'Johnny Depp', 'Anne Hathaway', 'Helena Bonh

In [256]:
print(rr)

{'Finding Nemo': ['Albert Brooks', 'Ellen DeGeneres', 'Alexander Gould', 'Willem Dafoe', 'Brad Garrett'], 'Dancer in the Dark': ['Björk', 'Catherine Deneuve', 'David Morse', 'Peter Stormare', 'Joel Grey'], 'Pirates of the Caribbean: The Curse of the Black Pearl': ['Johnny Depp', 'Geoffrey Rush', 'Orlando Bloom', 'Keira Knightley', 'Jack Davenport'], 'Kill Bill: Vol. 1': ['Uma Thurman', 'Lucy Liu', 'Vivica A. Fox', 'Daryl Hannah', 'David Carradine'], 'Jarhead': ['Jamie Foxx', 'Scott MacDonald', 'Lucas Black', 'Peter Sarsgaard'], '9 Songs': ['Kieran OBrien', 'Margo Stilley', 'Courtney Taylor-Taylor', 'Alex Kapranos', 'Guy Garvey'], 'The Simpsons Movie': ['Dan Castellaneta', 'Julie Kavner', 'Nancy Cartwright', 'Yeardley Smith', 'Hank Azaria'], 'Eternal Sunshine of the Spotless Mind': ['Jim Carrey', 'Kate Winslet', 'Kirsten Dunst', 'Mark Ruffalo', 'Elijah Wood'], "Pirates of the Caribbean: Dead Man's Chest": ['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Bill Nig